In [66]:
import pandas as pd
from census import Census
from us import states
from shapely.geometry import Point
import geopandas as gp
from scipy.stats import ttest_ind
import rtree
pd.set_option('display.max_columns', 500)

In [67]:
df = pd.read_csv('../other_data/selfsufficiencyandcosts.csv')
df.head()

,Family Type,Adult(s),Infant(s),Preshooler(s),Schoolager(s),Teenager(s),State,Year,All Families Table #,County,Housing Costs,Child Care Costs,Food Costs,Transportation Costs,Health Care Costs,Miscellaneous costs,Taxes,Earned Income Tax Credit (-),Child Care Tax Credit (-),Child Tax Credit (-),Hourly Self-Sufficiency Wage,Monthly Self-Sufficiency Wage,Annual Self-Sufficiency Wage,Emergency Savings
0,a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,1,Alameda County,"$2,322","$2,312",$939,$324,$545,$644,"$1,538",$0,($100),($333),$23.27,"$8,191","$98,296",$168
1,a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,2,Alpine County,$940,"$1,278",$887,$523,$620,$425,$743,$0,($100),($333),$14.15,"$4,982","$59,784",$97
2,a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,3,Amador County,"$1,055","$1,429","$1,037",$531,$620,$467,$862,$0,($100),($333),$15.82,"$5,568","$66,814",$108
3,a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,4,Butte County,$992,"$1,542",$816,$524,$620,$449,$812,$0,($100),($333),$15.12,"$5,323","$63,873",$104
4,a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,5,Calaveras County,$902,"$1,287","$1,121",$537,$620,$447,$804,$0,($100),($333),$15.01,"$5,285","$63,414",$103


In [68]:
all_counties = (
    gp.read_file(
        "../other_data/ca_counties_simplified.json"
    )
)
all_counties.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,01,005,00161528,0500000US01005,01005,Barbour,06,2292144656,50538698,"POLYGON ((-85.657668 31.880275, -85.587344 31...."
1,01,023,00161537,0500000US01023,01023,Choctaw,06,2365869837,19144469,"POLYGON ((-88.47322699999999 31.893856, -88.43..."
2,01,035,00161543,0500000US01035,01035,Conecuh,06,2201948618,6643480,"POLYGON ((-86.90589899999999 31.753035, -86.90..."
3,01,051,00161551,0500000US01051,01051,Elmore,06,1601762124,99965171,"POLYGON ((-86.37497399999999 32.75358, -86.336..."
4,01,065,00161558,0500000US01065,01065,Hale,06,1667907107,32423356,"POLYGON ((-87.71570899999999 33.006824, -87.42..."


In [69]:
ca_counties = all_counties.loc[all_counties['STATEFP'] == '06']
ca_counties.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
18,06,001,01675839,0500000US06001,06001,Alameda,06,1909616630,216916717,"POLYGON ((-122.315150304886 37.8968435739156, ..."
19,06,005,01675841,0500000US06005,06005,Amador,06,1539933576,29470568,"POLYGON ((-121.027507 38.508292, -120.885067 3..."
20,06,013,01675903,0500000US06013,06013,Contra Costa,06,1857310903,225193562,"POLYGON ((-122.269666188168 38.0603362895779, ..."
21,06,023,01681908,0500000US06023,06023,Humboldt,06,9241251740,1254039383,"POLYGON ((-124.065520896835 41.46473946493821,..."
22,06,037,00277283,0500000US06037,06037,Los Angeles,06,10510588451,1794793532,"(POLYGON ((-118.604415 33.478552, -118.538624 ..."


In [70]:
df['NAME'] = df['County'].str.replace(' County', '')
df.head()

,Family Type,Adult(s),Infant(s),Preshooler(s),Schoolager(s),Teenager(s),State,Year,All Families Table #,County,Housing Costs,Child Care Costs,Food Costs,Transportation Costs,Health Care Costs,Miscellaneous costs,Taxes,Earned Income Tax Credit (-),Child Care Tax Credit (-),Child Tax Credit (-),Hourly Self-Sufficiency Wage,Monthly Self-Sufficiency Wage,Annual Self-Sufficiency Wage,Emergency Savings,NAME
0,a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,1,Alameda County,"$2,322","$2,312",$939,$324,$545,$644,"$1,538",$0,($100),($333),$23.27,"$8,191","$98,296",$168,Alameda
1,a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,2,Alpine County,$940,"$1,278",$887,$523,$620,$425,$743,$0,($100),($333),$14.15,"$4,982","$59,784",$97,Alpine
2,a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,3,Amador County,"$1,055","$1,429","$1,037",$531,$620,$467,$862,$0,($100),($333),$15.82,"$5,568","$66,814",$108,Amador
3,a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,4,Butte County,$992,"$1,542",$816,$524,$620,$449,$812,$0,($100),($333),$15.12,"$5,323","$63,873",$104,Butte
4,a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,5,Calaveras County,$902,"$1,287","$1,121",$537,$620,$447,$804,$0,($100),($333),$15.01,"$5,285","$63,414",$103,Calaveras


In [71]:
ca_counties_merged = ca_counties.merge(df, on='NAME')

In [72]:
ca_counties_merged

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,Family Type,Adult(s),Infant(s),Preshooler(s),Schoolager(s),Teenager(s),State,Year,All Families Table #,County,Housing Costs,Child Care Costs,Food Costs,Transportation Costs,Health Care Costs,Miscellaneous costs,Taxes,Earned Income Tax Credit (-),Child Care Tax Credit (-),Child Tax Credit (-),Hourly Self-Sufficiency Wage,Monthly Self-Sufficiency Wage,Annual Self-Sufficiency Wage,Emergency Savings
0,06,001,01675839,0500000US06001,06001,Alameda,06,1909616630,216916717,"POLYGON ((-122.315150304886 37.8968435739156, ...",a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,1,Alameda County,"$2,322","$2,312",$939,$324,$545,$644,"$1,538",$0,($100),($333),$23.27,"$8,191","$98,296",$168
1,06,005,01675841,0500000US06005,06005,Amador,06,1539933576,29470568,"POLYGON ((-121.027507 38.508292, -120.885067 3...",a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,3,Amador County,"$1,055","$1,429","$1,037",$531,$620,$467,$862,$0,($100),($333),$15.82,"$5,568","$66,814",$108
2,06,013,01675903,0500000US06013,06013,Contra Costa,06,1857310903,225193562,"POLYGON ((-122.269666188168 38.0603362895779, ...",a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,7,Contra Costa County,"$2,339","$2,280",$894,$542,$602,$666,"$1,684",$0,($100),($333),$24.36,"$8,573","$102,877",$181
3,06,023,01681908,0500000US06023,06023,Humboldt,06,9241251740,1254039383,"POLYGON ((-124.065520896835 41.46473946493821,...",a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,12,Humboldt County,$956,"$1,514",$798,$518,$620,$441,$787,$0,($100),($333),$14.77,"$5,200","$62,404",$100
4,06,037,00277283,0500000US06037,06037,Los Angeles,06,10510588451,1794793532,"(POLYGON ((-118.604415 33.478552, -118.538624 ...",a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,19,Los Angeles County,"$1,663","$1,956",$853,$570,$414,$546,"$1,152",$0,($100),($333),$19.09,"$6,720","$80,642",$121
5,06,047,00277288,0500000US06047,06047,Merced,06,5012175320,112509475,"POLYGON ((-121.226804 37.134774, -120.964685 3...",a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,24,Merced County,$790,"$1,428",$785,$540,$597,$414,$713,$0,($100),($333),$13.73,"$4,833","$57,998",$95
6,06,067,00277298,0500000US06067,06067,Sacramento,06,2499039655,76217822,"POLYGON ((-121.593273 38.313089, -121.521641 3...",a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,34,Sacramento County,"$1,055","$1,651",$803,$555,$525,$459,$839,$0,($100),($333),$15.49,"$5,454","$65,449",$106
7,06,085,00277307,0500000US06085,06085,Santa Clara,06,3344209919,33276429,"POLYGON ((-122.11516140516 37.4662797321484, -...",a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,43,Santa Clara County,"$2,522","$2,354",$920,$525,$534,$686,"$1,808",$0,($100),($333),$25.33,"$8,915","$106,981",$251
8,06,115,00277322,0500000US06115,06115,Yuba,06,1636913850,31057914,"POLYGON ((-121.62376 39.295621, -121.611177 39...",a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,58,Yuba County,$881,"$1,529",$731,$528,$620,$429,$755,$0,($100),($333),$14.32,"$5,039","$60,467",$98
9,06,045,00277287,0500000US06045,06045,Mendocino,06,9081426673,962946204,"POLYGON ((-124.02325190922 40.00127634042889, ...",a2i0p1s1t0,2,0,1.0,1.0,0.0,CA,2018,23,Mendocino County,"$1,033","$1,502","$1,040",$522,$620,$472,$874,$0,($100),($333),$15.99,"$5,629","$67,552",$109


In [73]:
ca_counties_merged.shape

(58, 34)

In [74]:
dropped = ca_counties_merged.drop(columns=["COUNTYNS", "AFFGEOID", "LSAD", "ALAND", 'AWATER','Adult(s)', "Infant(s)", "Preshooler(s)", "Schoolager(s)","Teenager(s)", "All Families Table #", "Earned Income Tax Credit (-)", "Child Tax Credit (-)", "Child Care Tax Credit (-)"])

In [75]:
renamed = dropped.rename(index=str, columns={"Housing Costs": "Housing", "Child Care Costs" : "Childcare", "Transportation Costs" : "Transport", "Health Care Costs" : "Healthcare", "Miscellaneous costs" : "Misc", "Hourly Self Sufficiency"})

,STATEFP,COUNTYFP,GEOID,NAME,geometry,Family Type,State,Year,County,Housing Costs,Child Care Costs,Food Costs,Transportation Costs,Health Care Costs,Miscellaneous costs,Taxes,Hourly Self-Sufficiency Wage,Monthly Self-Sufficiency Wage,Annual Self-Sufficiency Wage,Emergency Savings
0,06,001,06001,Alameda,"POLYGON ((-122.315150304886 37.8968435739156, ...",a2i0p1s1t0,CA,2018,Alameda County,"$2,322","$2,312",$939,$324,$545,$644,"$1,538",$23.27,"$8,191","$98,296",$168
1,06,005,06005,Amador,"POLYGON ((-121.027507 38.508292, -120.885067 3...",a2i0p1s1t0,CA,2018,Amador County,"$1,055","$1,429","$1,037",$531,$620,$467,$862,$15.82,"$5,568","$66,814",$108
2,06,013,06013,Contra Costa,"POLYGON ((-122.269666188168 38.0603362895779, ...",a2i0p1s1t0,CA,2018,Contra Costa County,"$2,339","$2,280",$894,$542,$602,$666,"$1,684",$24.36,"$8,573","$102,877",$181
3,06,023,06023,Humboldt,"POLYGON ((-124.065520896835 41.46473946493821,...",a2i0p1s1t0,CA,2018,Humboldt County,$956,"$1,514",$798,$518,$620,$441,$787,$14.77,"$5,200","$62,404",$100
4,06,037,06037,Los Angeles,"(POLYGON ((-118.604415 33.478552, -118.538624 ...",a2i0p1s1t0,CA,2018,Los Angeles County,"$1,663","$1,956",$853,$570,$414,$546,"$1,152",$19.09,"$6,720","$80,642",$121


In [76]:
with open('../data/selfsufficiency.json', 'w') as f:
    f.write(dropped.to_json())